In [0]:
from google.colab import drive
import numpy as np
import pandas as pd
from collections import Counter
import os
import statistics as stats
import json
from collections import defaultdict

In [0]:
drive.mount('/content/drive')
path = os.getcwd()


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
{'mintempC': '1', 'maxtempF': '48', 'sunHour': '9.8', 'mintempF': '34', 'maxtempC': '9', 'hourly': [{'windspeedKmph': '15', 'FeelsLikeF': '35', 'winddir16Point': 'ENE', 'FeelsLikeC': '2', 'DewPointC': '0', 'windspeedMiles': '10', 'DewPointF': '33', 'HeatIndexF': '41', 'cloudcover': '0', 'HeatIndexC': '5', 'precipMM': '0.0', 'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.net/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png'}], 'WindGustMil

In [0]:
def processing(data):
  
  request = data['data']['request']
  data = data['data']['weather']
  df_hourly = pd.DataFrame()
  #print(len(data))
  for i in range(len(data)):
   df1 = pd.DataFrame(data = data[i]['hourly'])
   df_hourly = pd.concat([df_hourly,df1], ignore_index = True)
  #print(df_hourly.shape)
  '''for i in data[0].keys():
   df[i] = "0"
  print(df.shape)
  del df['hourly']
  del df['astronomy']'''
  columns = list(data[0].keys())
  #print(len(columns))
  columns.remove ('hourly')
  columns.remove ('astronomy')
  df_Astronomy = pd.DataFrame()
  for i in range(len(data)):
    df_temp = pd.DataFrame(data = data[i]['astronomy'])
    df_temp = pd.concat([df_temp]*24 , ignore_index = True)
    df_Astronomy = pd.concat([df_Astronomy,df_temp] , ignore_index = True)
  df_request = pd.DataFrame(data = request[0], index = [0])
  df_request = pd.concat([df_request]*len(df_hourly.index) , ignore_index = True)
  

  def conversion(columns,data):           #conversion is a function to convert the values apart from 'hourly' and 'astronomy' into a usable DataFrame

    flag = 0
    variable = []
    dict1 = defaultdict(list)
    for k in columns:
      for i in range (len(data)):
        variable.append(data[i][k])
    #print(variable)
    #print("The length of list of original values: ", len(variable))
    #print(len(data))
    list2=[]
    for i in variable:
      for j in range(24):
        list2.append(i)
    #print("The length of list of original values after duplication: ", len(list2))
    for i in columns:
      for j in range(24*len(data)):
        dict1[i].append(list2[flag])
        flag = flag + 1
    #print("The length of dictionary of duplicated values: ", len(dict1))
    return(dict1)

  columns_df = {}
  columns_df = conversion(columns, data)    
  df_values = pd.DataFrame(data = columns_df)
  
  df_month = pd.DataFrame()
  df_month = pd.concat([df_values,df_hourly,df_Astronomy,df_request], axis = 1)
  #print(df_month.shape)
  return(df_month)

In [0]:
def weather_processing(year , dataframe):
  
  
  path = os.getcwd()
  df_temp = pd.DataFrame()
  dir_list = [d for d in os.listdir(path + '/drive/My Drive/ML Project/weather') if os.path.isdir(os.path.join(path + '/drive/My Drive/ML Project/weather', d))]
  for i in dir_list:
   for j in range(12):
     file = open(path+'/drive/My Drive/ML Project/weather/{}/{}-{}.json'.format(i,year,(j+1)))
     data = json.load(file)
     df_temp = processing(data)
     dataframe = pd.concat([dataframe,df_temp],ignore_index = True)
  return(dataframe)  
  
df = pd.DataFrame()
df = weather_processing(2016 , df)
df = weather_processing(2017 , df)

  

(263160, 41)
mintempC             False
maxtempF             False
sunHour              False
mintempF             False
maxtempC             False
totalSnow_cm         False
date                 False
uvIndex              False
windspeedKmph        False
FeelsLikeF           False
winddir16Point       False
FeelsLikeC           False
DewPointC            False
windspeedMiles       False
DewPointF            False
HeatIndexF           False
cloudcover           False
HeatIndexC           False
precipMM             False
weatherIconUrl       False
WindGustMiles        False
pressure             False
WindGustKmph         False
weatherDesc          False
visibility           False
weatherCode          False
tempC                False
tempF                False
WindChillF           False
WindChillC           False
winddirDegree        False
humidity             False
time                 False
moon_phase           False
moonrise             False
moonset              False
sunset         

In [0]:
col_list = ['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity', 'date', 'time', 'query']
df = df[col_list]
print(df.shape)
df['airport'] = df['query'].str[:3]
df = df.drop(['query'], axis = 1)
df.shape


(263160, 15)


(263160, 15)

In [0]:
print(df.shape)
print("A", df.isna().any())
df.dropna()
df.shape
print(len(data))

df.to_csv(path + '/drive/My Drive/ML Project/Weather.csv', index = False)




(263160, 15)
A windspeedKmph    False
winddirDegree    False
weatherCode      False
precipMM         False
visibility       False
pressure         False
cloudcover       False
DewPointF        False
WindGustKmph     False
tempF            False
WindChillF       False
humidity         False
date             False
time             False
airport          False
dtype: bool
31


In [0]:
dataframe = pd.read_csv(path + '/drive/My Drive/ML Project/Weather.csv')
dataframe.shape
df1 = pd.DataFrame()
print (dataframe.shape)
print(dataframe.isna().any())
dataframe.dropna()
print(dataframe.shape)

(263160, 15)
windspeedKmph    False
winddirDegree    False
weatherCode      False
precipMM         False
visibility       False
pressure         False
cloudcover       False
DewPointF        False
WindGustKmph     False
tempF            False
WindChillF       False
humidity         False
date             False
time             False
airport          False
dtype: bool
(263160, 15)
